In [1]:
import requests

url = "https://drexel0.sharepoint.com/:x:/r/sites/ChangLab/Shared%20Documents/General/Group/Data/Gigafactory%20and%20Mining%20Data/SPGlobal_MetalsAndMiningProperties-Combined-Dec-2024.xlsx?d=w5f13853f2ff744ca8d824eecbca8f245&csf=1&web=1&e=ICjBVN"


requests.get(url, allow_redirects=True)


<Response [403]>